<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#About" data-toc-modified-id="About-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>About</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Data-import" data-toc-modified-id="Data-import-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data import</a></span><ul class="toc-item"><li><span><a href="#Data" data-toc-modified-id="Data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Data</a></span><ul class="toc-item"><li><span><a href="#train/test-data" data-toc-modified-id="train/test-data-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>train/test data</a></span></li><li><span><a href="#supplemental-data" data-toc-modified-id="supplemental-data-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>supplemental data</a></span></li><li><span><a href="#sample-submission" data-toc-modified-id="sample-submission-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>sample submission</a></span></li></ul></li></ul></li><li><span><a href="#Exploratory-data-analysis" data-toc-modified-id="Exploratory-data-analysis-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Exploratory data analysis</a></span></li><li><span><a href="#model-training" data-toc-modified-id="model-training-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>model training</a></span><ul class="toc-item"><li><span><a href="#data-preprocessing" data-toc-modified-id="data-preprocessing-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>data preprocessing</a></span></li><li><span><a href="#add-features" data-toc-modified-id="add-features-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>add features</a></span></li><li><span><a href="#simple-model" data-toc-modified-id="simple-model-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>simple model</a></span><ul class="toc-item"><li><span><a href="#ideas" data-toc-modified-id="ideas-5.3.1"><span class="toc-item-num">5.3.1&nbsp;&nbsp;</span>ideas</a></span></li><li><span><a href="#implementation" data-toc-modified-id="implementation-5.3.2"><span class="toc-item-num">5.3.2&nbsp;&nbsp;</span>implementation</a></span></li></ul></li></ul></li></ul></div>

# About

Data Description

You are provided with daily historical sales data. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge.
File descriptions

    train.csv - the training set. Daily historical data from January 2013 to October 2015.
    test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
    sample_submission.csv - a sample submission file in the correct format.
    items.csv - supplemental information about the items/products.
    item_categories.csv  - supplemental information about the items categories.
    shops.csv- supplemental information about the shops.

Data fields

    ID - an Id that represents a (Shop, Item) tuple within the test set
    shop_id - unique identifier of a shop
    item_id - unique identifier of a product
    item_category_id - unique identifier of item category
    item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
    item_price - current price of an item
    date - date in format dd/mm/yyyy
    date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
    item_name - name of item
    shop_name - name of shop
    item_category_name - name of item category

This dataset is permitted to be used for any purpose, including commercial use.


# Setup

In [9]:
run_on_kaggle = False

if run_on_kaggle:
    input_path = '/kaggle/input/'
    data_path = input_path + "competitive-data-science-predict-future-sales/"
    output_path = ""
else:
    input_path = '../data/'
    data_path = input_path
    output_path = ""

In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(input_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

../data/items.csv
../data/item_categories.csv
../data/sales_train.csv
../data/sample_submission.csv
../data/shops.csv
../data/test.csv


# Data import

## Data

### train/test data

In [41]:
# train.csv - the training set. Daily historical data from January 2013 to October 2015.
sales_train = pd.read_csv(data_path + "sales_train.csv")

# test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
test = pd.read_csv(data_path + "test.csv")

In [42]:
display(sales_train.head())
display(test.head())

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


### supplemental data

In [31]:
# items.csv - supplemental information about the items/products.
items = pd.read_csv(data_path + "items.csv")

# item_categories.csv  - supplemental information about the items categories.
item_categories = pd.read_csv(data_path + "item_categories.csv")

# shops.csv- supplemental information about the shops.
shops = pd.read_csv(data_path + "shops.csv")

In [32]:
display(items.head())
display(item_categories.head())
display(shops.head())

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


### sample submission

Submissions are evaluated by root mean squared error (RMSE). True target values are clipped into [0,20] range.

In [16]:
# sample_submission.csv - a sample submission file in the correct format.
sample_submission = pd.read_csv(data_path + "sample_submission.csv")

In [17]:
sample_submission

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5


# Exploratory data analysis

In [76]:
# keep a clean copy of the train data
train = sales_train.copy()

In [78]:
# convert to datetime 
train["date"] = pd.to_datetime(train["date"], format="%d.%m.%Y")

In [46]:
def calculate_revenue(df: pd.DataFrame) -> float:
    # some items are returned (item_cnt_day = -1.0); those need to be substracted
    return sum(df["item_price"]*df["item_cnt_day"])

In [73]:
revenue_per_shop_and_month = (
    train
    .groupby(["shop_id", "date_block_num"])
    .apply(lambda x: calculate_revenue(x))
    .rename("total_revenue_per_shop_and_month")
    .reset_index()
)

In [79]:
train = pd.merge(train, revenue_per_shop_and_month, how="left")

In [80]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,total_revenue_per_shop_and_month
0,2013-01-02,0,59,22154,999.00,1.0,1633431.00
1,2013-01-03,0,25,2552,899.00,1.0,5376478.13
2,2013-01-05,0,25,2552,899.00,-1.0,5376478.13
3,2013-01-06,0,25,2554,1709.05,1.0,5376478.13
4,2013-01-15,0,25,2555,1099.00,1.0,5376478.13
...,...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0,7633958.00
2935845,2015-10-09,33,25,7460,299.00,1.0,7633958.00
2935846,2015-10-14,33,25,7459,349.00,1.0,7633958.00
2935847,2015-10-22,33,25,7440,299.00,1.0,7633958.00


In [48]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue_per_shop
0,2013-01-02,0,59,22154,999.00,1.0,6.637370e+06
1,2013-01-03,0,25,2552,899.00,1.0,3.238207e+06
2,2013-01-05,0,25,2552,899.00,-1.0,4.404964e+07
3,2013-01-06,0,25,2554,1709.05,1.0,3.014085e+07
4,2013-01-15,0,25,2555,1099.00,1.0,4.053965e+07
...,...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0,NaN
2935845,2015-10-09,33,25,7460,299.00,1.0,NaN
2935846,2015-10-14,33,25,7459,349.00,1.0,NaN
2935847,2015-10-22,33,25,7440,299.00,1.0,NaN


In [49]:
train[train["shop_id"] == 25]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue_per_shop
1,2013-01-03,0,25,2552,899.00,1.0,3.238207e+06
2,2013-01-05,0,25,2552,899.00,-1.0,4.404964e+07
3,2013-01-06,0,25,2554,1709.05,1.0,3.014085e+07
4,2013-01-15,0,25,2555,1099.00,1.0,4.053965e+07
5,2013-01-10,0,25,2564,349.00,1.0,3.816043e+07
...,...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0,NaN
2935845,2015-10-09,33,25,7460,299.00,1.0,NaN
2935846,2015-10-14,33,25,7459,349.00,1.0,NaN
2935847,2015-10-22,33,25,7440,299.00,1.0,NaN


In [33]:
train["item_id"].nunique()

21807

In [34]:
train["shop_id"].nunique()

60

In [35]:
train[(train["shop_id"] == 5) & (train["item_id"] == 5037)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
1953995,21.09.2014,20,5,5037,2599.0,1.0
2150561,29.11.2014,22,5,5037,2599.0,1.0
2288630,28.12.2014,23,5,5037,1999.0,1.0
2288631,20.12.2014,23,5,5037,1999.0,1.0
2335446,02.01.2015,24,5,5037,1999.0,1.0
2335447,07.01.2015,24,5,5037,1999.0,1.0
2618926,29.05.2015,28,5,5037,1299.0,1.0
2704068,28.06.2015,29,5,5037,1499.0,1.0
2719247,05.07.2015,30,5,5037,1499.0,1.0
2810661,14.08.2015,31,5,5037,1499.0,1.0


In [36]:
train["shop_id"].nunique() * train["item_id"].nunique()

1308420

In [37]:
train["shop_id"].nunique()

60

In [38]:
train["item_id"].nunique()

21807

In [39]:
test["shop_id"].nunique()

42

In [40]:
test["item_id"].nunique()

5100

In [22]:
test[(test["shop_id"] == 5) & (test["item_id"] == 5037)]

,ID,shop_id,item_id
0,0,5,5037


In [23]:
test["shop_id"].nunique() * test["item_id"].nunique()

214200

In [21]:
pd.merge(test, sample_submission, on="ID")

,ID,shop_id,item_id,item_cnt_month
0,0,5,5037,0.5
1,1,5,5320,0.5
2,2,5,5233,0.5
3,3,5,5232,0.5
4,4,5,5268,0.5
...,...,...,...,...
214195,214195,45,18454,0.5
214196,214196,45,16188,0.5
214197,214197,45,15757,0.5
214198,214198,45,19648,0.5


# model training

## data preprocessing

In [89]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,total_revenue_per_shop_and_month
0,2013-01-02,0,59,22154,999.00,1.0,1633431.00
1,2013-01-03,0,25,2552,899.00,1.0,5376478.13
2,2013-01-05,0,25,2552,899.00,-1.0,5376478.13
3,2013-01-06,0,25,2554,1709.05,1.0,5376478.13
4,2013-01-15,0,25,2555,1099.00,1.0,5376478.13
...,...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0,7633958.00
2935845,2015-10-09,33,25,7460,299.00,1.0,7633958.00
2935846,2015-10-14,33,25,7459,349.00,1.0,7633958.00
2935847,2015-10-22,33,25,7440,299.00,1.0,7633958.00


## add features

## simple model

### ideas

 * Hierachical time-series prediction:
  * model per store
  * model per item
  * model per date
* models to try:
 * Gradient Boosting Machine
 * XGBoost
* play around with lag
 * (df[NUMERIC_COLUMN] - df.groupby(DATE_COLUMN)[NUMERIC_COLUMN].shift(-1)) and agg features (df.groupby(DATE_COLUMN)[NUMERIC_COLUMN].agg(std))
* clip the test data!
 * predictions = numpy.clip(predictions, 0 ,20)

### implementation

In [82]:
from sklearn.ensemble import RandomForestRegressor

In [84]:
from sklearn.datasets import make_regression

In [83]:
regr = RandomForestRegressor(max_depth=2, random_state=0)

In [85]:
X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)

In [86]:
regr.fit(X, y)

C:\Users\Benjamin\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [87]:
print(regr.predict([[0, 0, 0, 0]]))

[-2.50699856]


In [92]:
y

array([ 49.82290745,   4.87728597,  11.91487464,  19.75307803,
        23.60700003, -29.98278439,  -3.18275233,  81.61798896,
       -85.87627478,  66.296663  , -70.00907918, -25.41289672,
       -40.341885  , -50.80958638,  11.69850317, -26.06617118,
       -10.09289693, -16.81772822, -18.84474658, -37.62388016,
         6.90439112,  95.27998841,  14.08815923,  37.63961941,
        61.23802841,  -7.46245465,  42.37309217,  89.99454831,
        47.61800511,  14.11477091, -29.85246699, -19.3463247 ,
        27.69980617, -12.73102933,   4.69818239, -15.54546881,
        81.10896741, -42.73831876, -43.50103947, -13.81361736,
        55.72588193,   3.73964726,  35.17541383,  -4.68693299,
       -40.02555297, -54.23600024, -41.92953467,  37.56912025,
       -40.71853261,  43.95479823, -15.74495171, -42.13327197,
         2.38253778,   0.17626991,  12.60675473, -31.41170946,
       -29.41293009, -20.37016177,   3.09183413, -40.10546979,
       -26.63729755, -63.60114181, -66.77520312,  -8.00

In [93]:
X.shape

(100, 4)

In [88]:
X

array([[ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ],
       [ 1.86755799, -0.97727788,  0.95008842, -0.15135721],
       [-0.10321885,  0.4105985 ,  0.14404357,  1.45427351],
       [ 0.76103773,  0.12167502,  0.44386323,  0.33367433],
       [ 1.49407907, -0.20515826,  0.3130677 , -0.85409574],
       [-2.55298982,  0.6536186 ,  0.8644362 , -0.74216502],
       [ 2.26975462, -1.45436567,  0.04575852, -0.18718385],
       [ 1.53277921,  1.46935877,  0.15494743,  0.37816252],
       [-0.88778575, -1.98079647, -0.34791215,  0.15634897],
       [ 1.23029068,  1.20237985, -0.38732682, -0.30230275],
       [-1.04855297, -1.42001794, -1.70627019,  1.9507754 ],
       [-0.50965218, -0.4380743 , -1.25279536,  0.77749036],
       [-1.61389785, -0.21274028, -0.89546656,  0.3869025 ],
       [-0.51080514, -1.18063218, -0.02818223,  0.42833187],
       [ 0.06651722,  0.3024719 , -0.63432209, -0.36274117],
       [-0.67246045, -0.35955316, -0.81314628, -1.7262826 ],
       [ 0.17742614, -0.